<a href="https://colab.research.google.com/github/DaveCub/ML/blob/main/CNN_numbers_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- import libraries --------------------------------------------------
from numpy import mean
from numpy import std
from numpy import argmax
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import load_img
#from tensorflow.keras.utils.image_dataset_from_directory
from tensorflow.keras.preprocessing.image import img_to_array
#import os
#from os.path import join as path_join
from pathlib import Path

# ---- load train and test dataset --------------------------------------
def load_dataset():
	(trainX, trainY), (testX, testY) = mnist.load_data()   # load dataset
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))  # reshape dataset to have a single channel
	testX = testX.reshape((testX.shape[0], 28, 28, 1))     
	trainY = to_categorical(trainY)                        # one hot encode target values
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

#----- scale pixels  -------------------------------------------------------
def prep_pixels(train, test):
	train_norm = train.astype('float32')  # convert from integers to floats
	test_norm = test.astype('float32')
	train_norm = train_norm / 255.0       # normalize to range 0-1
	test_norm = test_norm / 255.0
	return train_norm, test_norm          # return normalized images

#---- define cnn model --------------------------------------------------------
def define_model():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	# compile model
	opt = SGD(learning_rate=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

# ----- evaluate a model using k-fold cross-validation ----------------------
def evaluate_model(dataX, dataY, n_folds=2):           # n-folds define the number of cycles
 trainX, trainY, testX, testY = load_dataset()         # load dataset
 trainX, testX = prep_pixels(trainX, testX)            # scale pixels
 scores, histories = list(), list()                    # initialize a empty list
 kfold = KFold(n_folds, shuffle=True, random_state=1)  # prepare cross validation
 for train_ix, test_ix in kfold.split(trainX):         # enumerate splits
  model = define_model()                               # define model
  trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]           # select rows for train and test
  history = model.fit(trainX, trainY, epochs=1, batch_size=32, validation_data=(testX, testY), verbose=0)   # fit model
  _, acc = model.evaluate(testX, testY, verbose=0)     # evaluate model
  print('> %.3f' % (acc * 100.0))
  scores.append(acc)                                   # stores scores
  histories.append(history)
 model.save('final_model.h5') 
 return scores, histories

# -------- plot diagnostic learning curves -----------------------------------
def summarize_diagnostics(histories):
 for i in range(len(histories)):
  plt.subplot(2, 1, 1)                 # plot loss
  plt.title('Cross Entropy Loss')
  plt.plot(histories[i].history['loss'], color='blue', label='train')
  plt.plot(histories[i].history['val_loss'], color='orange', label='test')
  plt.subplot(2, 1, 2)                 # plot accuracy
  plt.title('Classification Accuracy')
  plt.plot(histories[i].history['accuracy'], color='blue', label='train')
  plt.plot(histories[i].history['val_accuracy'], color='orange', label='test')
 plt.show()

# ----------- summarize model performance ----------------------------------
def summarize_performance(scores):
 print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))   # print summary
 plt.boxplot(scores)  # box and whisker plots of results
 plt.show()

# --------- load and prepare the new image -----------------------------------
def load_image(filename):
 data_folder = Path("sample_data/")
 filename=data_folder/filename
 #print(filename)
 img = load_img(filename, grayscale=True, target_size=(28, 28)) # load the image
 img = img_to_array(img)                                        # convert to array
 img = img.reshape(1, 28, 28, 1)                                # reshape into a single sample with 1 channel
 img = img.astype('float32')                                    # prepare pixel data
 img = img / 255.0
 return img


#------ run the test for evaluating a model --------------------------
def run_evaluation():
 trainX, trainY, testX, testY = load_dataset()        # load dataset
 trainX, testX = prep_pixels(trainX, testX)   	      # prepare pixel data
 scores, histories = evaluate_model(trainX, trainY)   # evaluate model
 # summarize_diagnostics(histories)                     # learning curves
 # summarize_performance(scores)                        # summarize estimated performance
 # model = load_model('final_model.h5')


# ----- load an image and predict the class --------------------------------
def predict():
 # model = load_model('final_model.h5')
 #img = load_image('sample_image.png')   # load the image
 img = load_image('Sin_titulo.png')   # load the image
 predict_value = model.predict(img)     # predict the class
 digit = argmax(predict_value)
 print(digit)
 
#-----------------------------------------------------------------------------
#--------------------------- MAIN CODE --------------------------------------- 
#-----------------------------------------------------------------------------
# run_evaluation()                      # fit and evaluate model accuracy
model = load_model('final_model.h5')  # for avoid the need to fit every time

# new images in grayscale, handwritten digit centered aligned, size is 28×28 pixels (square) 
predict()                             # run example of 1 photo




1/1 [==============================] - 0s 120ms/step
3
